In [22]:
import random

In [23]:
class Plant:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.active = False
        self.grow_chance = 0.5  # Начальная вероятность роста

    def can_grow(self, time_of_day):
        return False

    def adapt(self, world):
        # Адаптация: уменьшаем вероятность роста при высокой плотности растений
        plant_density = sum(1 for entity in world.entities if isinstance(entity, Plant)) / (world.width * world.height)
        self.grow_chance = max(0.1, 0.5 * (1 - plant_density))  # Чем выше плотность, тем ниже шанс роста

    def grow(self, world, time_of_day):
        self.adapt(world)  # Адаптируемся к среде перед ростом
        if not self.can_grow(time_of_day):
            return []
        neighbors = [(self.x+dx, self.y+dy) for dx, dy in [(-1,0), (1,0), (0,-1), (0,1)]]
        valid_neighbors = []
        for nx, ny in neighbors:
            if 0 <= nx < world.width and 0 <= ny < world.height and not world.grid[nx][ny]:
                valid_neighbors.append((nx, ny))
        if valid_neighbors and random.random() < self.grow_chance:
            nx, ny = random.choice(valid_neighbors)
            return [self.__class__(nx, ny)]
        return []

In [24]:
class Time:
    def __init__(self):
        self.cycle = 0  # Счётчик тиков
        self.times_of_day = ["morning", "day", "evening", "night"]
        
    def get_time(self):
        return self.times_of_day[self.cycle % 4]
    
    def tick(self):
        self.cycle += 1


In [25]:
class Dandelion(Plant):
    def can_grow(self, time_of_day):
        return time_of_day == "day"

class Amanita(Plant):
    def can_grow(self, time_of_day):
        return time_of_day == "night"

class Orange(Plant):
    def can_grow(self, time_of_day):
        return time_of_day in ["morning", "evening"]

In [26]:
class Animal:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.hunger = 0
        self.group_size = 1
        self.move_chance = 0.5  # Начальная вероятность движения
        self.attack_chance = 0.3  # Начальная вероятность атаки

    def adapt(self, world):
        # Адаптация: изменяем поведение в зависимости от плотности еды и врагов
        food_count = sum(1 for entity in world.entities if isinstance(entity, Plant))
        enemy_count = sum(1 for entity in world.entities if isinstance(entity, Animal) and not isinstance(entity, self.__class__))
        total_cells = world.width * world.height
        food_density = food_count / total_cells
        enemy_density = enemy_count / total_cells
        # Если еды мало, увеличиваем подвижность
        self.move_chance = min(0.9, 0.5 + (0.5 - food_density))
        # Если много врагов, увеличиваем агрессию
        self.attack_chance = min(0.5, 0.3 + enemy_density)

    def act(self, world, time_of_day):
        pass

    def move(self, world):
        neighbors = [(self.x+dx, self.y+dy) for dx, dy in [(-1,0), (1,0), (0,-1), (0,1)]]
        valid_neighbors = [(nx, ny) for nx, ny in neighbors if 0 <= nx < world.width and 0 <= ny < world.height]
        if valid_neighbors and random.random() < self.move_chance:
            self.x, self.y = random.choice(valid_neighbors)

In [27]:
class Chimpanzee(Animal):
    def act(self, world, time_of_day):
        self.adapt(world)  # Адаптируемся к среде
        if time_of_day == "night":
            return
        self.hunger += 1
        eat_chance = 0.8 if time_of_day == "morning" else 0.2 if time_of_day == "evening" else 0.5
        if random.random() < eat_chance:
            for dx, dy in [(-1,0), (1,0), (0,-1), (0,1)]:
                nx, ny = self.x + dx, self.y + dy
                if 0 <= nx < world.width and 0 <= ny < world.height:
                    entity = world.grid[nx][ny]
                    if isinstance(entity, Dandelion):
                        world.grid[nx][ny] = None
                        self.hunger = max(0, self.hunger - 2)
                        break
        if self.hunger > 5 and random.random() < self.attack_chance:
            for dx, dy in [(-1,0), (1,0), (0,-1), (0,1)]:
                nx, ny = self.x + dx, self.y + dy
                if 0 <= nx < world.width and 0 <= ny < world.height:
                    entity = world.grid[nx][ny]
                    if isinstance(entity, Chimpanzee):
                        world.grid[nx][ny] = None
        self.move(world)

class Orangutan(Animal):
    def act(self, world, time_of_day):
        self.adapt(world)  # Адаптируемся к среде
        if time_of_day in ["day", "night"]:
            return
        self.hunger += 1
        if random.random() < self.move_chance:
            self.move(world)
        for dx, dy in [(-1,0), (1,0), (0,-1), (0,1)]:
            nx, ny = self.x + dx, self.y + dy
            if 0 <= nx < world.width and 0 <= ny < world.height:
                entity = world.grid[nx][ny]
                if isinstance(entity, (Orange, Amanita, Chimpanzee)):
                    world.grid[nx][ny] = None
                    self.hunger = max(0, self.hunger - 3)
                    break

In [28]:
class World:
    def __init__(self, width, height):
        self.width = width
        self.height = height
        self.grid = [[None for _ in range(height)] for _ in range(width)]
        self.entities = []

    def add_entity(self, entity):
        if not self.grid[entity.x][entity.y]:
            self.grid[entity.x][entity.y] = entity
            self.entities.append(entity)

    def step(self, time_of_day):
        new_entities = []
        for entity in self.entities[:]:
            if isinstance(entity, Plant):
                new_plants = entity.grow(self, time_of_day)
                new_entities.extend(new_plants)
            elif isinstance(entity, Animal):
                entity.act(self, time_of_day)
        for entity in new_entities:
            self.add_entity(entity)

    def display(self):
        symbols = {Dandelion: 'D', Amanita: 'A', Orange: 'O', Chimpanzee: 'C', Orangutan: 'An', None: '.'}
        for i in range(self.width):
            row = [symbols[type(self.grid[i][j])] if self.grid[i][j] else '.' for j in range(self.height)]
            print(' '.join(row))
        print()

In [29]:
def main():
    world = World(10, 10)
    time = Time()

    for _ in range(10):
        x, y = random.randint(0, 9), random.randint(0, 9)
        world.add_entity(Dandelion(x, y))
        x, y = random.randint(0, 9), random.randint(0, 9)
        world.add_entity(Amanita(x, y))
        x, y = random.randint(0, 9), random.randint(0, 9)
        world.add_entity(Orange(x, y))
    for _ in range(5):
        x, y = random.randint(0, 9), random.randint(0, 9)
        world.add_entity(Chimpanzee(x, y))
        x, y = random.randint(0, 9), random.randint(0, 9)
        world.add_entity(Orangutan(x, y))

    for step in range(20):
        time_of_day = time.get_time()
        print(f"Step {step + 1}, Time: {time_of_day}")
        world.step(time_of_day)
        world.display()
        time.tick()

if __name__ == "__main__":
    main()

Step 1, Time: morning
D . . . A . O . D .
. O O . . . O . . .
. . O . O . . . . D
D An O . . . O . . .
. . O An . . . . A .
. . . . . A . . D D
. C . O A . . C . An
. An A . O . . C . .
. . . A . . . . . .
A . C . C O D . D D

Step 2, Time: day
D . . . A . O . D .
. O O . . . O . . .
. . O . O . . . D D
D An O . . . O . . .
. . O An . . . . A D
. . . . . A . . D D
. C . O A . . C . An
. An A . O . . C . .
. . . A . . D . D .
A . C . C O D . D D

Step 3, Time: evening
D O . . A O O . D .
. O O . . . O . . .
. O O . O . . . D D
D An O . . O O . . .
. . . An . . . . . D
. O . . . A . . D D
. C . O A . . C . An
. An A . O . . C . .
. . . A . . D . D .
A . C . C O . . D D

Step 4, Time: night
D O . . A O O . D .
. O O . . . O . . .
. O O . O . . . D D
D An O . . O O . A .
. . . An . . . . . D
. O . . . A . . D D
. C . O A . . C . An
. An A . O . . C . .
. . . A . . D . D .
A . C . C O . . D D

Step 5, Time: morning
D O O . A O O . D .
. O O . . . O . . .
. . O . O . . . D D
D An O . . O O O